<a href="https://colab.research.google.com/github/Betsabeh/Drug_traget_Deep/blob/main/ChermBert_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers datasets tokenizers torch accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 67.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 11.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 58.6 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [3]:
#import package
import torch
import pandas as pd
import numpy as np
from transformers import AutoTokenizer, AutoModel


In [4]:
# For ChemBERTa
tokenizer = AutoTokenizer.from_pretrained("DeepChem/ChemBERTa-10M-MLM") # Or other ChemBERTa variants
model = AutoModel.from_pretrained("DeepChem/ChemBERTa-10M-MLM")
# move the model to GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
print("model move to", device)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.27k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/631 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/6.96k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/52.0 [00:00<?, ?B/s]

...

added_tokens.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/420 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/13.7M [00:00<?, ?B/s]

Some weights of RobertaModel were not initialized from the model checkpoint at DeepChem/ChemBERTa-10M-MLM and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


model move to cuda


In [5]:
smiles_string = ["CCOc1c(Cl)cc(NC(=O)CCc2cn(C)c3cc(C)ccc23)cc1Cl", "CCC"]
print("smile lenght=",len(smiles_string))
inputs = tokenizer(smiles_string, return_tensors="pt", padding=True, truncation=True).to(model.device)
print("Input IDs (first SMILES example):")
print(inputs['input_ids'][0])
print("\nAttention Mask (first SMILES example):")
print(inputs['attention_mask'][0])
print(f"\nVocabulary size: {len(tokenizer.vocab)}")
print(f"Max input length for this model: {tokenizer.model_max_length}")

with torch.no_grad():
    outputs = model(**inputs)
    #print(outputs)

embeddings = outputs.last_hidden_state # Or pooler_output depending on your needs
print("embeding size=",embeddings.shape) # Should be (batch_size, sequence_length, hidden_size)

smile lenght= 2
Input IDs (first SMILES example):
tensor([12, 16, 16, 19, 15, 20, 15, 17, 16, 18, 15, 15, 17, 23, 16, 17, 22, 19,
        18, 16, 16, 15, 21, 15, 25, 17, 16, 18, 15, 26, 15, 15, 17, 16, 18, 15,
        15, 15, 21, 26, 18, 15, 15, 20, 16, 13], device='cuda:0')

Attention Mask (first SMILES example):
tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
       device='cuda:0')

Vocabulary size: 593
Max input length for this model: 512
embeding size= torch.Size([2, 46, 384])
